## Command line options

It is extremely easy for you to pass command line options to a SoS script.

### Optional arguments

Any variable assignment prefixed with `parameter:` can accept values from command  line. The format of such lines are

```
# comment
parameter: var_name = default_value
```

The default value can be number, string, list of string, or expressions that return values of these types. Other types can be used as long as they can be converted to these types from user-provided values. For example

```
# path to tool gatk
parameter: gatk_path = '~/bin/GATK'
```

defines a variable `gatk_path` with default value `'~/bin/GATK'`.

```
# A list of sample names
parameter: sample_names=[]
```

defines a variable `sample_name` with default value `[]`, and

```python
# path to gatk
parameter: gatk_path = CONFIG['gatk_path']
```
uses `gatk_path` from a YAML-based configuration file (specified from command line using option `-c`) as default value. You can set default value for `CONFIG` inside SoS script via:

```python
# path to gatk
parameter: gatk_path = CONFIG.get('gatk_path', '/default/path/to/gatk')
```

so that if configuration files are not supplied the default value `/default/path/to/gatk` will be used.

The default values not only determines the values of variable when they are not specified from command line or configuration files, but also determines the type of input these parameters accept. For example, with the above definitions for command arguments `--gatk_path` and `--sample_names`, you can pass values to these variables from command line,

```bash
sos run myscript --gatk_path /path/to/gatk --sample_names A1 A2 A3
```

A list will be passed to `sample_names` even if only a single value is provided (e.g. `sample_names=['A1']` for `--sample_name A1`).
Attempts to pass more than one values (a list) to `gatk_path` (e.g. `--gatk_path /path1 /path2`) will trigger an error.

Note that boolean values can be specified from command line as `--param` for `True` and `--no-param` for `False`. It is also allowed to use `--gatk-path` in addition to `--gatk_path` for parameter `--gatk_path`.

### Required arguments

In cases where there is no suitable default values and/or command line arguments are mandatary, you can list the type of arguments (e.g. `int`, `bool`, `str`, `list` of strings) in place of default values. For example, if an integer parameter `cutoff` is required, you can define it as

```python
# cutoff value
parameter: cutoff = int
```

This will force the users to provide an integer to this parameter. You can do the same for lists but SoS assumes that you need a **list of strings**. For example, the following definition

```python
# input bam files
parameter: bam_files = list
```

request a list of strings from command line. SoS will return a list even if only one value is provided.

### Redefinition of `parameter`

A very important property of the `parameter` directive is that **parameter definition is only effective for undefined variable**. That is to say, the parameter statement will be ignored if a parameter has been specified in some way, most likely as a variable in a step.

For example, the following workflow 

```sos
[default_1]
parameter: cutoff=10
sh:
  do something with ${cutoff}

[default_2]
sh:
  do something else

[batch]
for cutoff in range(10):
    sos_run('default')
```

would be called in a number of ways

```bash
sos run myscript 
sos run myscript --cutoff 5
sos run myscript batch
```

The first command uses default value `10` for parameter `cutoff`, the second command assigns value `5` from command line, and the third command uses a series of values `0`, `1`, .... Because `cutoff` is already defined for workflow `default`, the parameter statement will be ignored.